<a href="https://colab.research.google.com/github/zhe0/aia_nlp/blob/main/Lab1-4_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%%capture
!gdown 1CJnqY8itczW9TgJRM_IP19eousGEQP0r # omicron.csv，若檔名有錯記得改名
!gdown 1RIAyeFRRXRoSq0U9lNWz1R9HkbGjCadV # tweet_dataset.csv
# !gdown 1qDkQl92rylA6YpCD1qmL3drvfVrrOnHD # country_vaccinations.csv
# !gdown 1PRdsIGs6fkC940434O6-A_E61Ih_HLh_ # country_vaccinations_by_manufacturer.csv

!gdwon 1aDDGuW5IfHbXZ5_vjz2daGotnaFDuBBS # 維吾爾新聞文本
!pip3 install -U transformers

In [2]:
import re
import numpy as np
import pandas as pd
import os, json
import torch
import time 
import datetime
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, get_linear_schedule_with_warmup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = sorted(list(set(stopwords.words('english'))))
print('nltk provide ',len(stop_words),' stopwords') #個人覺得太少，可以自己補充

nltk provide  179  stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
elif torch.backends.mps.is_available():
    device = torch.device('cuda')
else:
    device = torch.device("cpu")
print(f'Using "{device}"')

Using "cuda"


#### before training, data prepare

In [4]:
df = pd.read_csv('tweet_dataset.csv')
df.head()

,textID,sentiment,author,text,old_text,aux_id,new_sentiment,selected_text
0,1956967341,empty,xoshayzers,i know i was listenin to bad habit earlier a...,@tiffanylue i know i was listenin to bad habi...,p1000000000,NaN,NaN
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh...waitin o...,c811396dc2,negative,headache
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...,Funeral ceremony...gloomy friday...,9063631ab1,negative,gloomy
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2a815f151d,positive,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,We want to trade with someone who has Houston...,@dannycastillo We want to trade with someone w...,82565a56d3,neutral,We want to trade with someone who has Houston ...


We only extract `text` and `new_sentiment`

In [5]:
df_ = df.dropna().reset_index(drop=True) 
lb=df_['new_sentiment'].unique().tolist() #['negative', 'neutral', 'positive']

# 原始欄位有sentiment可以選擇
df_['lb']=df_['new_sentiment'].apply(lambda x:[ 1 if x==i else 0 for i in lb])
df_=df_.drop(['textID','sentiment','author','old_text','aux_id','selected_text'], axis=1)
df_

,text,new_sentiment,lb
0,Layin n bed with a headache ughhhh...waitin o...,negative,"[1, 0, 0]"
1,Funeral ceremony...gloomy friday...,negative,"[1, 0, 0]"
2,wants to hang out with friends SOON!,positive,"[0, 1, 0]"
3,We want to trade with someone who has Houston...,neutral,"[0, 0, 1]"
4,Re-pinging : why didn`t you go to prom? BC my ...,negative,"[1, 0, 0]"
...,...,...,...
27762,Ooo showing of your French skills!! lol Thing...,positive,"[0, 1, 0]"
27763,Happy Mothers Day All my love,positive,"[0, 1, 0]"
27764,Happy Mother`s Day to all the mommies out ther...,positive,"[0, 1, 0]"
27765,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY ...,neutral,"[0, 0, 1]"


#### Import BERT Tokenizer

In [6]:
model_name='bert-base-cased' # 中文可用 bert-base-chinese
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True) #英文可用

#### Preprocessing

In [7]:
# 前處理
url_pattern = re.compile(r'https?://\S+|www\.\S+') #去掉有類似網址部分

def prep(tweet):
  tweet = url_pattern.sub(r'', tweet)
  tweet = re.sub('\S*@\S*\s?', '', tweet)
  tweet = re.sub('\s+', ' ', tweet)
  tweet = re.sub("\'", "", tweet)
  # for s in stop_words: #想要去除stop word可嘗試，但不建議刪去
  #   tweet = re.sub(s, "", tweet)
  return tweet

df_['text']=df_['text'].apply(lambda x:prep(x))
df_

,text,new_sentiment,lb
0,Layin n bed with a headache ughhhh...waitin on...,negative,"[1, 0, 0]"
1,Funeral ceremony...gloomy friday...,negative,"[1, 0, 0]"
2,wants to hang out with friends SOON!,positive,"[0, 1, 0]"
3,We want to trade with someone who has Houston...,neutral,"[0, 0, 1]"
4,Re-pinging : why didn`t you go to prom? BC my ...,negative,"[1, 0, 0]"
...,...,...,...
27762,Ooo showing of your French skills!! lol Thing...,positive,"[0, 1, 0]"
27763,Happy Mothers Day All my love,positive,"[0, 1, 0]"
27764,Happy Mother`s Day to all the mommies out ther...,positive,"[0, 1, 0]"
27765,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY N...,neutral,"[0, 0, 1]"


#### Datasets and DataLoaders

In [8]:
# 把資料切割成八比二
test_size=int(len(df_)*0.2) #預計切出的test_size
df_train, df_eval=train_test_split(df_,test_size=test_size,stratify=df_["new_sentiment"])
print('training length and evaluation length', len(df_train), len(df_eval))

training length and evaluation length 22214 5553


In [9]:
batch_size=64 #一次送模組多少筆資料的批次量，不僅影響訓練速度，也影響辨識率的重要參數
def cvt2DataLoader(d,batch_size):
  p={'truncation':True,'padding':True, 'return_tensors':'pt', 'return_token_type_ids':True, 'return_attention_mask':True}
  enc=tokenizer.batch_encode_plus(d['text'].values.tolist(), **p)
  p=[enc['input_ids'],enc['token_type_ids'],enc['attention_mask'],
       torch.tensor(d['lb'].tolist()).float()]
  return DataLoader(TensorDataset(*p),batch_size=batch_size,shuffle=True)
dl_train=cvt2DataLoader(df_train,batch_size=batch_size)
dl_eval=cvt2DataLoader(df_eval,batch_size=batch_size)
print('training一圈資料需要次數',len(dl_train),len(dl_eval))

training一圈資料需要次數 348 87


### building model

In [10]:
# transformer 的文章分類模組，請檢查最後輸出「(classifier): Linear(in_features=768, out_features=3, bias=True)」
model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels = len(lb), 
    output_attentions = False,
    output_hidden_states = False
)
model

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### Optimizer and Scheduler

In [11]:
epochs = 5
lr = 2e-5
optimizer = AdamW(model.parameters(),lr = lr, eps=1e-8)

total_steps = len(dl_train) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps) #這行其實沒必要

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
import time, datetime, random

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

#### Training Loop

In [ ]:
criterion=torch.nn.BCEWithLogitsLoss()
model.to(device)

training_stats = []
total_t0 = time.time()
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(dl_train):
        if step % 40 == 0:
            elapsed = format_time(time.time() - t0)
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dl_train), elapsed))
        
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0],
              'token_type_ids': batch[1],
              'attention_mask': batch[2],
              'return_dict': True}
        
        model.zero_grad(set_to_none=True) #optimizer.zero_grad()，儘管作用一樣，但清model比較安全

        result = model(**inputs)
        loss = criterion(out_code, batch[3])
        loss = result.loss
        logits = result.logits
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        avg_train_loss = total_train_loss / len(dl_train)            

    training_time = format_time(time.time() - t0)

    print("")
    print(" Average training loss: {0:.2f}".format(avg_train_loss))
    print(" Training epcoh took: {:}".format(training_time))


======== Epoch 1 / 5 ========
Training...
  Batch     0  of    882.    Elapsed: 0:00:00.
  Batch    40  of    882.    Elapsed: 0:00:08.
  Batch    80  of    882.    Elapsed: 0:00:16.
  Batch   120  of    882.    Elapsed: 0:00:24.
  Batch   160  of    882.    Elapsed: 0:00:32.
  Batch   200  of    882.    Elapsed: 0:00:40.
  Batch   240  of    882.    Elapsed: 0:00:48.
  Batch   280  of    882.    Elapsed: 0:00:56.
  Batch   320  of    882.    Elapsed: 0:01:04.
  Batch   360  of    882.    Elapsed: 0:01:12.
  Batch   400  of    882.    Elapsed: 0:01:20.
  Batch   440  of    882.    Elapsed: 0:01:28.
  Batch   480  of    882.    Elapsed: 0:01:36.
  Batch   520  of    882.    Elapsed: 0:01:45.
  Batch   560  of    882.    Elapsed: 0:01:53.
  Batch   600  of    882.    Elapsed: 0:02:01.
  Batch   640  of    882.    Elapsed: 0:02:09.
  Batch   680  of    882.    Elapsed: 0:02:17.
  Batch   720  of    882.    Elapsed: 0:02:25.
  Batch   760  of    882.    Elapsed: 0:02:33.
  Batch   800  of

In [ ]:
print("Running Validation...")
t0 = time.time()

model.eval()

total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():        
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask,
                       labels=b_labels,
                       return_dict=True)
        loss = result.loss
        logits = result.logits
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Running Validation...
  Accuracy: 0.01
  Validation Loss: 0.01
  Validation took: 0:00:00
  Accuracy: 0.02
  Validation Loss: 0.01
  Validation took: 0:00:00
  Accuracy: 0.03
  Validation Loss: 0.01
  Validation took: 0:00:00
  Accuracy: 0.03
  Validation Loss: 0.02
  Validation took: 0:00:00
  Accuracy: 0.04
  Validation Loss: 0.03
  Validation took: 0:00:00
  Accuracy: 0.05
  Validation Loss: 0.03
  Validation took: 0:00:00
  Accuracy: 0.06
  Validation Loss: 0.04
  Validation took: 0:00:00
  Accuracy: 0.07
  Validation Loss: 0.05
  Validation took: 0:00:01
  Accuracy: 0.08
  Validation Loss: 0.05
  Validation took: 0:00:01
  Accuracy: 0.08
  Validation Loss: 0.06
  Validation took: 0:00:01
  Accuracy: 0.09
  Validation Loss: 0.07
  Validation took: 0:00:01
  Accuracy: 0.10
  Validation Loss: 0.07
  Validation took: 0:00:01
  Accuracy: 0.11
  Validation Loss: 0.07
  Validation took: 0:00:01
  Accuracy: 0.12
  Validation Loss: 0.08
  Validation took: 0:00:01
  Accuracy: 0.13
  Validat

#### Prediction

In [ ]:
df_vax = pd.read_csv('omicron.csv')
df_vax.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1465693385088323591,Abaris,Hants,"Would appear on Blogger (did originally), but...",2009-09-16 14:30:32,2880,4369,26907,False,2021-11-30 14:45:08,"@SkyNews ""Told you I'd be Back!"" #OMICRON “Odi...",['OMICRON'],Twitter Web App,0,0,False
1,1465693062999412746,GFTs ,Lalaland,There's a field somewhere beyond all doubt and...,2019-12-28 14:29:13,165,583,21152,False,2021-11-30 14:43:52,Someone told me this in October #Omicron https...,['Omicron'],Twitter for Android,0,0,False
2,1465690116442279942,Herbie Finkle (Cozy),NaN,help me find me frens ❤️,2021-07-10 09:40:57,114,393,2339,False,2021-11-30 14:32:09,Glad to see the public schoolkids are wrapping...,['COVID'],Twitter Web App,0,1,False
3,1465689607165591552,Electrical Review,United Kingdom,Electrical Review is a monthly journal aimed a...,2009-05-21 08:32:19,20759,2321,739,False,2021-11-30 14:30:08,#Automation systems have become increasingly c...,['Automation'],Hootsuite Inc.,0,0,False
4,1465688203709464578,BingX Academy 🔑,NaN,BingX (Prev. Bingbon) is the world's top crypt...,2013-12-31 02:57:32,17134,8,31,False,2021-11-30 14:24:33,"🟢 If u think Omicron is a FUD, you LONG📈\n🔴 If...",NaN,Twitter Web App,2,2,False
